%cd '/content/drive/My Drive/sentence_similarity'

In [0]:
import csv
import pickle
import gensim
from text2digits import text2digits
from gensim.models import doc2vec
from collections import namedtuple
from gensim.models import Word2Vec
from scipy import spatial
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import scipy
from scipy import stats
from nltk.chunk import  tree2conlltags



In [0]:
def read_dataset(filename):
    sentences1 = []
    sentences2 = []
    all_sentences = []
    similarity_score = []
    with open(filename, 'r', encoding='utf-8') as f:

        data = csv.reader(f)
        for row in data:
            if row[5] != '' and row[6] != '':
                sentences1.append(row[5])
                sentences2.append(row[6])
                all_sentences.append(row[5])
                all_sentences.append(row[6])

                similarity_score.append(float(float(row[4])/5))

    return sentences1,sentences2,all_sentences,similarity_score

In [0]:
filename="/content/drive/My Drive/sentence_similarity/sts-train.csv"
sentences1,sentences2,all_sentences,similarity_score=read_dataset(filename)

In [28]:
print(sentences1[:10])

print(sentences2[:10])

print(similarity_score[:10])

['A plane is taking off.', 'A man is playing a large flute.', 'A man is spreading shreded cheese on a pizza.', 'Three men are playing chess.', 'A man is playing the cello.', 'Some men are fighting.', 'A man is smoking.', 'The man is playing the piano.', 'A man is playing on a guitar and singing.', 'A person is throwing a cat on to the ceiling.']
['An air plane is taking off.', 'A man is playing a flute.', 'A man is spreading shredded cheese on an uncooked pizza.', 'Two men are playing chess.', 'A man seated is playing the cello.', 'Two men are fighting.', 'A man is skating.', 'The man is playing the guitar.', 'A woman is playing an acoustic guitar and singing.', 'A person throws a cat on the ceiling.']
[1.0, 0.76, 0.76, 0.52, 0.85, 0.85, 0.1, 0.32, 0.44000000000000006, 1.0]


In [0]:
def preprocess(sentences1, sentences2):
    # function to convert it into lower case
    sentences3 = []
    sentences4 = []
    for sentence in sentences1:
        sentences3.append(sentence.lower())
    for sentence in sentences2:
        sentences4.append(sentence.lower())
    return sentences3, sentences4


sentences1, sentences2 = preprocess(sentences1, sentences2)


def contractionsEN(sentences):
    # copywrite source: https://github.com/cipriantruica/CATS/blob/master/cats/nlplib/static.py
    # slightly modified dict than the source
    # takes words like he'll and convert it into he will
    # tokenize the input by splitting on spaces
    result = []
    for sentence in sentences:
        sentence = sentence.replace(".,'[]", " ")
        words = sentence.split()

        contractions_en2 = {
            "'s": " is",
            "'ve": " have",
            "'d": " had",
            "ain't": "am not",
            "aren't": "are not",
            "can't": "cannot",
            "can't've": "cannot have",
            "'cause": "because",
            "could've": "could have",
            "couldn't": "could not",
            "couldn't've": "could not have",
            "didn't": "did not",
            "doesn't": "does not",
            "don't": "do not",
            "hadn't": "had not",
            "hadn't've": "had not have",
            "hasn't": "has not",
            "haven't": "have not",
            "he'd": "he had",
            "he'd've": "he would have",
            "he'll": "he will",
            "he'll've": "he will have",
            "he's": "he has",
            "how'd": "how did",
            "how'd'y": "how do you",
            "how'll": "how will",
            "how's": "how has",
            "i'd": "i had",
            "i'd've": "i would have",
            "i'll": "i will",
            "i'll've": "i will have",
            "i'm": "i am",
            "i've": "i have",
            "isn't": "is not",
            "it'd": "it had",
            "it'd've": "it would have",
            "it'll": "it will",
            "it'll've": "it will have",
            "it's": "it is",
            "let's": "let us",
            "ma'am": "madam",
            "mayn't": "may not",
            "might've": "might have",
            "mightn't": "might not",
            "mightn't've": "might not have",
            "must've": "must have",
            "mustn't": "must not",
            "mustn't've": "must not have",
            "needn't": "need not",
            "needn't've": "need not have",
            "o'clock": "of the clock",
            "oughtn't": "ought not",
            "oughtn't've": "ought not have",
            "shan't": "shall not",
            "sha'n't": "shall not",
            "shan't've": "shall not have",
            "she'd": "she had",
            "she'd've": "she would have",
            "she'll": "she will",
            "she'll've": "she will have",
            "she's": "she is",
            "should've": "should have",
            "shouldn't": "should not",
            "shouldn't've": "should not have",
            "so've": "so have",
            "so's": "so is",
            "that'd": "that had",
            "that'd've": "that would have",
            "that's": "that is",
            "there'd": "there would",
            "there'd've": "there would have",
            "there's": "there is",
            "they'd": "they would",
            "they'd've": "they would have",
            "they'll": "they will",
            "they'll've": "they will have",
            "they're": "they are",
            "they've": "they have",
            "to've": "to have",
            "wasn't": "was not",
            "we'd": "we would",
            "we'd've": "we would have",
            "we'll": "we will",
            "we'll've": "we will have",
            "we're": "we are",
            "we've": "we have",
            "weren't": "were not",
            "what'll": "what will",
            "what'll've": "what will have",
            "what're": "what are",
            "what's": "what is",
            "what've": "what have",
            "when's": "when has",
            "when've": "when have",
            "where'd": "where did",
            "where's": "where has",
            "where've": "where have",
            "who'll": "who will",
            "who'll've": "who will have",
            "who's": "who has",
            "who've": "who have",
            "why's": "why has",
            "why've": "why have",
            "will've": "will have",
            "won't": "will not",
            "won't've": "will not have",
            "would've": "would have",
            "wouldn't": "would not",
            "wouldn't've": "would not have",
            "y'all": "you all",
            "y'all'd": "you all would",
            "y'all'd've": "you all would have",
            "y'all're": "you all are",
            "y'all've": "you all have",
            "you'd": "you had",
            "you'd've": "you would have",
            "you'll": "you will",
            "you'll've": "you will have",
            "you're": "you are",
            "you've": "you have"
        }
        words2 = []
        for word in words:
            try:

                words2.append(contractions_en2[word])
            except:
                words2.append(word)
        sen = ' '.join(i for i in words2)
        result.append(sen)
    return result


sentences1 = contractionsEN(sentences1)
sentences2 = contractionsEN(sentences2)



In [0]:
def tokenize(sentences1, sentences2):
    # splits the sentence into words based on space
    words_sentences1 = []
    words_sentences2 = []
    for sentence in sentences1:
        words_sentences1.append(sentence.split())
    for sentence in sentences2:
        words_sentences2.append(sentence.split())
    return words_sentences1, words_sentences2


words_sentences1, words_sentences2 = tokenize(sentences1, sentences2)

In [31]:
print(words_sentences1[:10])


[['a', 'plane', 'is', 'taking', 'off.'], ['a', 'man', 'is', 'playing', 'a', 'large', 'flute.'], ['a', 'man', 'is', 'spreading', 'shreded', 'cheese', 'on', 'a', 'pizza.'], ['three', 'men', 'are', 'playing', 'chess.'], ['a', 'man', 'is', 'playing', 'the', 'cello.'], ['some', 'men', 'are', 'fighting.'], ['a', 'man', 'is', 'smoking.'], ['the', 'man', 'is', 'playing', 'the', 'piano.'], ['a', 'man', 'is', 'playing', 'on', 'a', 'guitar', 'and', 'singing.'], ['a', 'person', 'is', 'throwing', 'a', 'cat', 'on', 'to', 'the', 'ceiling.']]


In [0]:
def generate_postags(words_sentences1, words_sentences2):
    # takes as input a tokenized sentence and returns their NE tags
    ne_tags_sentences1 = []
    ne_tags_sentences2 = []
    for words in words_sentences1:
        ne_tags_sentences1.append(nltk.pos_tag(words))
    for words in words_sentences2:
        ne_tags_sentences2.append(nltk.pos_tag(words))
    return ne_tags_sentences1, ne_tags_sentences2


ne_tags_sentences1, ne_tags_sentences2 = generate_postags(words_sentences1, words_sentences2)

In [35]:
print(ne_tags_sentences1[:10])

[[('a', 'DT'), ('plane', 'NN'), ('is', 'VBZ'), ('taking', 'VBG'), ('off.', 'RP')], [('a', 'DT'), ('man', 'NN'), ('is', 'VBZ'), ('playing', 'VBG'), ('a', 'DT'), ('large', 'JJ'), ('flute.', 'NN')], [('a', 'DT'), ('man', 'NN'), ('is', 'VBZ'), ('spreading', 'VBG'), ('shreded', 'VBN'), ('cheese', 'NN'), ('on', 'IN'), ('a', 'DT'), ('pizza.', 'NN')], [('three', 'CD'), ('men', 'NNS'), ('are', 'VBP'), ('playing', 'VBG'), ('chess.', 'NNS')], [('a', 'DT'), ('man', 'NN'), ('is', 'VBZ'), ('playing', 'VBG'), ('the', 'DT'), ('cello.', 'NN')], [('some', 'DT'), ('men', 'NNS'), ('are', 'VBP'), ('fighting.', 'JJ')], [('a', 'DT'), ('man', 'NN'), ('is', 'VBZ'), ('smoking.', 'JJ')], [('the', 'DT'), ('man', 'NN'), ('is', 'VBZ'), ('playing', 'VBG'), ('the', 'DT'), ('piano.', 'NN')], [('a', 'DT'), ('man', 'NN'), ('is', 'VBZ'), ('playing', 'VBG'), ('on', 'IN'), ('a', 'DT'), ('guitar', 'NN'), ('and', 'CC'), ('singing.', 'NN')], [('a', 'DT'), ('person', 'NN'), ('is', 'VBZ'), ('throwing', 'VBG'), ('a', 'DT'), ('ca

In [0]:
def group_by_tag(ne_tags_sentences1, ne_tags_sentences2):
    res1 = []
    res2 = []
    for words in ne_tags_sentences1:
        tags = set()

        for i in range(0, len(words)):
            tags.add(words[i][1])
        tags = list(tags)
        sen_tag_pair = {}
        for i in range(0, len(tags)):
            temp = []
            for j in range(0, len(words)):
                if words[j][1] == tags[i]:
                    temp.append(words[j][0])
            sen_tag_pair[tags[i]] = temp

        res1.append(sen_tag_pair)

    for words in ne_tags_sentences2:
        tags = set()

        for i in range(0, len(words)):
            tags.add(words[i][1])
        tags = list(tags)
        sen_tag_pair = {}
        for i in range(0, len(tags)):
            temp = []
            for j in range(0, len(words)):
                if words[j][1] == tags[i]:
                    temp.append(words[j][0])
            sen_tag_pair[tags[i]] = temp

        res2.append(sen_tag_pair)
    return res1, res2


ne_grouped_sentence1, ne_grouped_sentence2 = group_by_tag(ne_tags_sentences1, ne_tags_sentences2)
ne_grouped_sentence1 = ne_grouped_sentence1[:5708]

In [37]:
print(ne_grouped_sentence1[:10])

[{'DT': ['a'], 'RP': ['off.'], 'NN': ['plane'], 'VBG': ['taking'], 'VBZ': ['is']}, {'DT': ['a', 'a'], 'JJ': ['large'], 'NN': ['man', 'flute.'], 'VBG': ['playing'], 'VBZ': ['is']}, {'VBN': ['shreded'], 'DT': ['a', 'a'], 'NN': ['man', 'cheese', 'pizza.'], 'VBG': ['spreading'], 'IN': ['on'], 'VBZ': ['is']}, {'VBP': ['are'], 'VBG': ['playing'], 'NNS': ['men', 'chess.'], 'CD': ['three']}, {'NN': ['man', 'cello.'], 'VBG': ['playing'], 'DT': ['a', 'the'], 'VBZ': ['is']}, {'VBP': ['are'], 'JJ': ['fighting.'], 'DT': ['some'], 'NNS': ['men']}, {'NN': ['man'], 'JJ': ['smoking.'], 'DT': ['a'], 'VBZ': ['is']}, {'NN': ['man', 'piano.'], 'VBG': ['playing'], 'DT': ['the', 'the'], 'VBZ': ['is']}, {'DT': ['a', 'a'], 'CC': ['and'], 'NN': ['man', 'guitar', 'singing.'], 'VBG': ['playing'], 'IN': ['on'], 'VBZ': ['is']}, {'DT': ['a', 'a', 'the'], 'NN': ['person', 'cat', 'ceiling.'], 'VBG': ['throwing'], 'TO': ['to'], 'IN': ['on'], 'VBZ': ['is']}]


In [0]:
def pair_similar_tags(ne_grouped_sentences1, ne_grouped_sentences2):
    # takes the ne tagged sentences and groups together the words that have the same NE tags

    matched_pairs = []
    for i in range(0, len(ne_grouped_sentence1)):
        result = {}
        for key, val in ne_grouped_sentence1[i].items():

            for key1, val1 in ne_grouped_sentence2[i].items():

                if key == key1:
                    res = []
                    res.append(val)
                    res.append(val1)
                    result[key] = res
        matched_pairs.append(result)
    return matched_pairs


matched_pairs = pair_similar_tags(ne_grouped_sentence1,ne_grouped_sentence2)


In [39]:
print(matched_pairs[:10])

[{'DT': [['a'], ['an']], 'RP': [['off.'], ['off.']], 'NN': [['plane'], ['air', 'plane']], 'VBG': [['taking'], ['taking']], 'VBZ': [['is'], ['is']]}, {'DT': [['a', 'a'], ['a', 'a']], 'NN': [['man', 'flute.'], ['man', 'flute.']], 'VBG': [['playing'], ['playing']], 'VBZ': [['is'], ['is']]}, {'VBN': [['shreded'], ['shredded']], 'DT': [['a', 'a'], ['a', 'an']], 'NN': [['man', 'cheese', 'pizza.'], ['man', 'cheese', 'pizza.']], 'VBG': [['spreading'], ['spreading']], 'IN': [['on'], ['on']], 'VBZ': [['is'], ['is']]}, {'VBP': [['are'], ['are']], 'VBG': [['playing'], ['playing']], 'NNS': [['men', 'chess.'], ['men', 'chess.']], 'CD': [['three'], ['two']]}, {'NN': [['man', 'cello.'], ['man', 'cello.']], 'VBG': [['playing'], ['playing']], 'DT': [['a', 'the'], ['a', 'the']], 'VBZ': [['is'], ['is']]}, {'VBP': [['are'], ['are']], 'JJ': [['fighting.'], ['fighting.']], 'NNS': [['men'], ['men']]}, {'NN': [['man'], ['man']], 'JJ': [['smoking.'], ['skating.']], 'DT': [['a'], ['a']], 'VBZ': [['is'], ['is']]}

In [0]:
def bag_of_words(words1, words2):
    # takes as an input two lists which are the words present in 2 sentences and combines them into a single list

    bag = []
    for word in words1:
        bag.append(word)
    for word in words2:
        bag.append(word)
    return bag


def skipgram_word2vec(bag):
    # takes two sentences(their bag of words representation)
    # converts each word into their word vector representation
    # uses the skipgram model

    model = gensim.models.Word2Vec([bag], min_count=1, window=3, sg=1)
    return model



In [42]:
def calculate_similarity(matched_pairs, word_sentences1, word_sentences2):
    # calculate the cosine similarity of the vector representation of 2 sentences
    cosine_similarity = []
    for i in range(0, len(words_sentences1)):
        bag = bag_of_words(word_sentences1[i], word_sentences2[i])
        model = skipgram_word2vec(bag)

        final = 0
        match = 0
        for key, val in matched_pairs[i].items():

            first_list = val[0]
            second_list = val[1]

            result = 0
            if len(first_list) > len(second_list):
                match += len(second_list)
                for word1 in second_list:
                    max = -100
                    for word2 in first_list:
                        similar = model.similarity(word1, word2)
                        if similar > max:
                            max = similar

                    result += max


            else:
                match += len(first_list)
                for word1 in first_list:
                    max = -100
                    for word2 in second_list:
                        similar = model.similarity(word1, word2)
                        if similar > max:
                            max = similar

                    result += max

            final += result
        try:
            temp1 = final / match
            cosine_similarity.append(temp1)
        except:
            cosine_similarity.append(0)

    return cosine_similarity


cos_sim = calculate_similarity(matched_pairs, words_sentences1, words_sentences2)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


In [43]:
print(cos_sim[:10])

[0.8152072742581368, 0.9999999701976776, 0.8865301108194722, 0.7830357909202575, 1.0000000099341075, 0.9999999403953552, 0.7656959872692823, 0.8600303182999293, 0.8804805977270007, 0.8496381919831038]


In [44]:
def generate_ne_tags(words_sentences1):
    # takes a tokenized sentence and generates the NE tags for it
    answer = []
    for sentence in words_sentences1:
        result = []
        sent = nltk.pos_tag(sentence)
        pattern = 'NP: {<DT>?<JJ>*<NN>}'
        cp = nltk.RegexpParser(pattern)
        cs = cp.parse(sent)
        iob_tagged = tree2conlltags(cs)
        for i in range(0, len(iob_tagged)):
            result.append((sentence[i], iob_tagged[i][-1]))
        answer.append(result)
    return answer


ne_tagged_sentences1 = generate_ne_tags(words_sentences1)
ne_tagged_sentences2 = generate_ne_tags(words_sentences2)


def group_ne_tagged(ne_tagged_sentences1):
    answer = []

    for i in range(0, len(ne_tagged_sentences1)):
        tags = set()
        for j in range(0, len(ne_tagged_sentences1[i])):
            tags.add(ne_tagged_sentences1[i][j][1])
        tags = list(tags)
        final_tag = {}
        for tag in tags:
            result = []
            for j in range(0, len(ne_tagged_sentences1[i])):
                if ne_tagged_sentences1[i][j][1] == tag:
                    result.append(ne_tagged_sentences1[i][j][0])
            final_tag[tag] = result
        answer.append(final_tag)

    return answer


grouped_sentence1 = group_ne_tagged(ne_tagged_sentences1)
grouped_sentence2 = group_ne_tagged(ne_tagged_sentences2)

matched_pairs2 = pair_similar_tags(grouped_sentence1, grouped_sentence2)
cos_sim2 = calculate_similarity(matched_pairs2, words_sentences1, words_sentences2)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


In [0]:
def bow_vec(sentences1, sentences2):
    # takes two sentences(their bag of words representation)
    # converts each word into their word vector representation
    # uses the continuous bag of words model

    docs = []
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
    for i, text in enumerate(sentences1):
        words = text.lower().split()
        tags = [i]
        docs.append(analyzedDocument(words, tags))
    model11 = doc2vec.Doc2Vec(docs, vector_size=50, min_count=1)

    docs2 = []
    analyzedDocument2 = namedtuple('AnalyzedDocument', 'words tags')
    for i, text in enumerate(sentences2):
        words = text.lower().split()
        tags = [i]
        docs2.append(analyzedDocument2(words, tags))
    model22 = doc2vec.Doc2Vec(docs2, vector_size=50, min_count=1)
    return model11, model22


model11, model22 = bow_vec(sentences1, sentences2)


In [0]:
def written_num():
    # program to convert thirty five to 35

    converter = text2digits.Text2Digits()
    num_similarity = []
    for i in range(0, len(sentences1)):
        sen1 = converter.convert(sentences1[i])
        sen2 = converter.convert(sentences2[i])
        num1 = 0
        num2 = 0
        for s in sen1.split():
            if s.isdigit():
                num1 += int(s)
        for s in sen2.split():
            if s.isdigit():
                num2 += int(s)

        if num1 != 0 and num2 != 0:
            num_similarity.append(min(num1, num2) / max(num1, num2))
        else:
            num_similarity.append(0)

    return num_similarity


num_sim = written_num()



In [47]:
def length_sentences(sentences1, sentences2):
    diff = []
    for i in range(0, len(sentences1)):
        diff.append(abs(len(sentences1[i]) - len(sentences2[i])))
    return diff


diff = length_sentences(sentences1, sentences2)


def bow_similarity():
    # input: sentences
    # output: tf-idf weighted vector of the sentence

    all_sentences.append(" ")
    model = Word2Vec(all_sentences, min_count=1, size=100)
    vocab = model.wv.vocab.keys()
    wordsInVocab = len(vocab)

    import numpy as np

    def sent_vectorizer(sent, model):
        sent_vec = np.zeros(100)
        numw = 0
        for w in sent:
            try:
                sent_vec = np.add(sent_vec, model[w])
                numw += 1
            except:
                pass
        return sent_vec / np.sqrt(sent_vec.dot(sent_vec))

    V = []
    for sentence in all_sentences:
        V.append(sent_vectorizer(sentence, model))

    from numpy import dot
    from numpy.linalg import norm

    results = [[0 for i in range(len(V))] for j in range(len(V))]

    for i in range(len(V) - 1):
        for j in range(i + 1, len(V)):
            results[i][j] = dot(V[i], V[j]) / norm(V[i]) / norm(V[j])

    # print(results)

    from sklearn.feature_extraction.text import TfidfTransformer
    tfidf = TfidfTransformer(norm="l2")
    tfidf.fit(results)
    # print("IDF:", tfidf.idf_)
    tf_idf_matrix = tfidf.transform(results)

    answer = tf_idf_matrix.todense()

    sparse_similarity = []
    evens = [x for x in range(len(all_sentences)) if x % 2 == 0]
    odds = [x for x in range(len(all_sentences)) if x % 2 != 0]

    for z in range(0, len(evens)):
        try:
            sparse_similarity.append(1 - spatial.distance.cosine(answer[evens[z]], answer[odds[z]]))
        except:
            continue

    return sparse_similarity


xyz = bow_similarity()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
with open("/content/drive/My Drive/sentence_similarity/train_features_v1.csv", 'w+', encoding='utf-8') as g:
    for i in range(0, len(sentences1)):


        # feature 1
        g.write(str(cos_sim[i]) + "\t")

        # features 2

        g.write(str(cos_sim2[i]) + "\t")

        # feature 3

        g.write(str(xyz[i]) + "\t")

        # feature 4

        g.write(str(num_sim[i]) + "\t")

        # for feature 5
        g.write(str(diff[i]) + "\t")


        #actual
        g.write(str(similarity_score[i])+"\t")
        g.write("\n")


In [0]:
df = pd.read_csv('/content/drive/My Drive/sentence_similarity/train_features_v1.csv', header=None, sep='\t')
df.rename(columns={0: 'ne_tagged', 1: 'pos_tagged', 2: 'bag_of_words', 3: 'numeric_sum', 4: 'diff_in_length',5: 'actual_similarity'}, inplace=True)
df.to_csv('/content/drive/My Drive/sentence_similarity/train_features.csv', index=False)


In [54]:
dataset = pd.read_csv("/content/drive/My Drive/sentence_similarity/train_features.csv")

X = dataset.drop('actual_similarity', axis=1)
y = dataset['actual_similarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)
X_train=X_train.values
X_train=np.nan_to_num(X_train)
y_train=y_train.values
y_train=np.nan_to_num(y_train)
X_test=np.nan_to_num(X_test.values)
y_test=np.nan_to_num(y_test.values)

def save_model(rf):



    pickle.dump(rf, open("/content/drive/My Drive/sentence_similarity/finalized_model.sav", 'wb'))

def load_model():
    loaded_model = pickle.load(open("/content/drive/My Drive/sentence_similarity/finalized_model.sav", 'rb'))
    return loaded_model


rf = RandomForestRegressor(n_estimators = 1024, random_state = 48,max_depth=8)
rf.fit(X_train,y_train)
save_model(rf)
model=load_model()
predictions =model.predict(X_test)

print(scipy.stats.pearsonr(predictions,y_test))

a=scipy.stats.pearsonr(predictions,y_test)
print("The pearson correlation Coefficient is ",a[0].__round__(6)*100)

(0.5386824939137813, 1.9425424695711184e-85)
The pearson correlation Coefficient is  53.8682
